<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/rating/classify_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run images through image rating classification pipeline
---
*Last Updated 13 September 2025*  
-Runs in Python 3 with Tensorflow 2.0-   

Use trained image classification model to add tags for image quality rating (bad, good) to EOL images.

Models were trained in [rating_train.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/rating/rating_train.ipynb). Confidence threshold for the best trained model was selected in [inspect_train_results.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/rating/inspect_train_results.ipynb).

We observed controversy among users assigning ratings to "good" images, and consensus for assigning ratings to "bad" images (Users were more conflicted on what they like than what they don't like). Model behavior matched this observation. In post-processing, keep only "bad" image quality predictions (model accuracy was high for this class) when confidence > 1.5. "Good" image quality predications are discarded (model accuracy was low for this class).

Finally, display tagging results on images to verify behavior is as expected.

***Models were trained in Python 2 and TF 1 in December 2020: MobileNet SSD v2 (Run 18, trained on 'good' and 'bad' classes) was trained for 12 hours to 10 epochs with Batch Size=16, Lr=0.001, Dropout=0.2.***

Notes:     
* Run code blocks by pressing play button in brackets on left
* Before you you start: change the runtime to "GPU" with "High RAM"
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

## Installs & Imports
---

In [ ]:
#@title Choose where to save results and set up environment
import os

# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]
print("Saving results ", save)

# Mount google drive to export image tagging file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type of classification pipeline
classif_type = "rating" #@param ["image_type", "rating"] {allow-input: true}

# Type in the path to your project wd in form field on right
basewd = "/content/drive/MyDrive/train" #@param ["/content/drive/MyDrive/train"] {allow-input: true}
basewd = basewd + '/' + classif_type

# Type in the folder that you want to contain TF2 files
folder = "results" #@param ["darknet"] {allow-input: true}
cwd = basewd + '/' + folder
print(f"\033[92m\nWorking directory set to: \n{cwd}\n\033[0m")

# Download helper_funcs folder
!pip3 -q install --upgrade gdown
!gdown  1TfpSLwbt6i0OMdbbjcTPVFBP6vYc3CY_
!tar -xzvf helper_funcs.tar.gz -C .

# Install requirements.txt
#!pip3 -q install -r requirements.txt

# Install compatible versions of numpy and tensorflow (Note: auto-restarts the runtime during first install. Need to run first two blocks twice to us new versions)
import tensorflow as tf
import numpy as np

if tf.__version__ != '2.17.0' or not np.__version__.startswith('1.26'):
    print("\033[93m\n\n\nThe first time you run this block, it will trigger an auto-restart of the Colab runtime to use newly installed versions. Re-run, then proceed with the remaining notebook steps to generate tags.\033[0m\n\n\n")
    !pip install --force-reinstall tensorflow==2.17.0 numpy==1.26.4 --no-cache-dir
    # Force restarts the runtime cleanly
    os.kill(os.getpid(), 9)
else:
    print("\033[92mSuccessfully installed and restarted to use compatible versions of tensorflow and numpy for generating image tags.\033[0m")

In [ ]:
#@title Choose saved model parameters (if using EOL model, defaults are already selected)
from setup import *

# Use EOL pre-trained model for object detection?
use_EOL_model = True #@param {type: "boolean"}

# Download EOL model if appropriate
if (use_EOL_model==True) & (os.path.exists("18.zip")==False):
    !gdown 1L-WqfuoQtPgqJzU8tDKjgsZC98M-68w9

# If using your own trained model, change values to match your trained model
module_selection = ("mobilenet_v2_1.0_224", 224) #@param ["(\"mobilenet_v2_1.0_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
dataset_labels = ["bad", "good"] #@param ["[\"bad\", \"good\"]"] {type:"raw", allow-input: true}
saved_models_folder = "saved_models" #@param ["train/saved_models/"] {allow-input: true}
saved_models_dir = basewd + '/' + saved_models_folder + '/'
TRAIN_SESS_NUM = "18" #@param ["18"] {allow-input: true}

# Set up directory structure
setup_dirs(cwd)
%cd $basewd

# Unpack EOL saved model
trained_model_dir = saved_models_dir + TRAIN_SESS_NUM + '/'
unpack_EOL_model(use_EOL_model, trained_model_dir, basewd, TRAIN_SESS_NUM, classif_type)

# Load saved model
module_selection, dataset_labels = get_model_info(TRAIN_SESS_NUM)
model, pixels, handle_base = load_saved_model(saved_models_dir, TRAIN_SESS_NUM, module_selection)

In [ ]:
#@title Import libraries

# For downloading and displaying images
from PIL import Image
import cv2
import imageio
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# For working with data
import numpy as np
from numpy import sum
import pandas as pd
from os import path
import csv
import itertools
from scipy.linalg import norm
# So URL's don't get truncated in display
pd.set_option('display.max_colwidth',1000)
pd.options.display.max_columns = None

# For measuring inference time
import time

# For image classification
import tensorflow as tf
print('\nTensorflow Version: %s' % tf.__version__)

# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# Import wrangle data helper funcs
from wrangle_data import *

# Define functions
# Set filename for saving classification results
def set_outpath(tags_file):
    outpath = cwd + '/' + tags_file + '.tsv'
    print("\nSaving results to: \n", outpath)

    return outpath

# Export results
def export_results(df, url, det_imclass, conf, i):
    # Define variables for export
    if 'ancestry' in df.columns:
        ancestry = df['ancestry'][i]
    else:
        ancestry = "NA"
    identifier = df['identifier'][i]
    dataObjectVersionID = df['dataObjectVersionID'][i]
    # Write row with results for each image
    results = [url, identifier, dataObjectVersionID, ancestry,
               det_imclass, conf]
    with open(outfpath, 'a') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(results)

## Generate tags: Run inference on EOL images & save results for tagging - Run 4X for batches A-D
---
Use 20K EOL image bundle to classify image quality rating as bad or good. Results are saved to [tags_file].tsv. Run this section 4 times (to make batches A-D) of 5K images each to incrementally save in case of Colab timeouts.

### Prepare classification functions and settings

In [ ]:
#@title Enter EOL image bundle and choose inference settings. Change **tags_file** for each batch A-D
%cd $cwd

# Load in EOL image bundle
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Squamata_20K_breakdown_000001.txt" #@param ["https://editors.eol.org/other_files/bundle_images/files/images_for_Squamata_20K_breakdown_000001.txt", "https://editors.eol.org/other_files/bundle_images/files/images_for_Coleoptera_20K_breakdown_000001.txt", "https://editors.eol.org/other_files/bundle_images/files/images_for_Anura_20K_breakdown_000001.txt", "https://editors.eol.org/other_files/bundle_images/files/images_for_Carnivora_20K_breakdown_000001.txt"] {allow-input: true}
df = read_datafile(bundle, sep='\t', header=0, disp_head=False)

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for 500 images"]
print("Run: ", run)

# Take 5k subset of bundle for running inference
# Change filename for each batch
tags_file = "rating_tags_tf2_d" #@param ["rating_tags_tf2_a", "rating_tags_tf2_b", "rating_tags_tf2_c", "rating_tags_tf2_d"] {allow-input: true}
outfpath = set_outpath(tags_file)

# Write header row of tagging file
print("\nClassification predictions for tags_file {} being saved to : \n{}\n".format(tags_file, outfpath))
if not os.path.isfile(outfpath):
    with open(outfpath, 'a') as out_file:
              tsv_writer = csv.writer(out_file, delimiter='\t')
              tsv_writer.writerow(["eolMediaURL", "identifier", \
                                   "dataObjectVersionID", "ancestry", \
                                   "imclass", "confidence"])

In [ ]:
#@title Run inference

# Define start and stop indices for EOL bundle images
start, stop, cutoff = set_start_stop(run, df)

# Run EOL bundle images through trained model
print(f"Running inference for images {start} : {stop} of EOL bundle: {bundle}\n")
run_inference(
    model=model,
    df=df,
    pixels=pixels,
    dataset_labels=dataset_labels,
    start=start,
    stop=stop,
    cutoff=cutoff,
    outfpath=outfpath,
    image_from_url=image_from_url,
    get_predict_info=get_predict_info,
    export_results=export_results)

## Post-process classification results
---
MobileNet SSD v2 confidence threshold (>1.5) for all 'bad' predictions was chosen in inspect_train_results.ipynb to minimize false detections and maximize dataset coverage. All 'good' predictions and any 'bad' predictions below the confidence threshold are discarded.

In [ ]:
#@title Use chosen confidence threshold (or EOL default)

# Adjust confidence threshold parameter
conf_thresh = 1.5 #@param ["1.5"] {type:"raw", allow-input: true}

# Combine tagging files for batches A-D
fpath =  os.path.splitext(tags_file)[0] # Get name of one tag file
base = cwd + '/' + fpath.rsplit('_',1)[0] + '_' # Remove lettered suffix to get basename
exts = ['a.tsv', 'b.tsv', 'c.tsv', 'd.tsv']
all_filenames = [base + e for e in exts] # List all tag filenames
df = pd.concat([pd.read_csv(f, sep='\t', header=0, na_filter = False) for f in all_filenames], ignore_index=True)
df[['confidence']] = df[['confidence']].apply(pd.to_numeric)

# Summarize combined results
print("Model predictions for Training Attempt {}, {}:".format(TRAIN_SESS_NUM, handle_base))
print("No. Images: {}\n{}".format(len(df), df[['eolMediaURL', 'imclass', 'confidence']].head()))

# Discard all predictions for 'good' or below confidence threshold
# (Final tag to keep -> predictions for 'bad' with confidence > 1.5)
idx_tokeep = df.index[(df.imclass == 'bad') & (df.confidence > conf_thresh)]
idx_todiscard = df.index.difference(idx_tokeep)
df.loc[idx_todiscard, 'imclass'] = 'NA'

# Write results to tsv
print("\nFinal tagging dataset after filtering predictions: \n", df[['eolMediaURL', 'imclass', 'confidence']].head())
outfpath = base + 'final.tsv'
print("\nSaving results to: \n", outfpath)
df.to_csv(outfpath, sep='\t', index=False)

## Display classification results on images
---

In [ ]:
#@title Adjust start index and display up to 50 images with their tags

# Adjust start index using slider
start = 0 #@param {type:"slider", min:0, max:5000, step:50}
stop = min((start+50), len(df))

# Loop through EOL image bundle to classify images and generate tags
for i, row in df.iloc[start:stop].iterrows():
    try:
        # Read in image from url
        url = df['eolMediaURL'][i]
        fn = str(i) + '.jpg'
        img, disp_img, _ = image_from_url(url, fn, pixels)

        # Get quality rating tag
        tag = df['imclass'][i]

        # Display progress message after each image is loaded
        print('Successfully loaded {} of {} images'.format(i+1, (stop-start)))

        # Show classification results for images
        # Only use to view predictions on <50 images at a time
        plot_image_results(i, disp_img, tag)

    except:
        print('Check if URL from {} is valid'.format(url))